In [2]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [3]:
%load_ext tensorboard

In [4]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())

8


In [9]:
vocab, index = {}, 1
vocab['<pad>'] = 0
for tone in tokens:
    if tone not in vocab:
        vocab[tone] = len(vocab)
        index += 1
vocab_size = len(vocab)

inverse_vocab = {index: token for token, index in vocab.items()}
print(vocab)
print(inverse_vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}
{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [10]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [27]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence, vocab_size, window_size=window_size, negative_samples=0)
  
print(positive_skip_grams)

[[5, 6], [2, 4], [3, 1], [2, 1], [4, 5], [6, 7], [1, 6], [4, 3], [5, 1], [6, 1], [7, 1], [4, 1], [5, 3], [2, 3], [1, 3], [6, 5], [7, 6], [3, 4], [3, 5], [1, 2], [1, 5], [3, 2], [5, 4], [1, 4], [1, 7], [4, 2]]


In [28]:
sentence = "The wide road shimmered in the hot sun"
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(5, 6): (in, hot)
(2, 4): (wide, shimmered)
(3, 1): (road, the)
(2, 1): (wide, the)
(4, 5): (shimmered, in)


In [29]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


2022-05-22 13:53:45.376443: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-22 13:53:45.376567: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
